In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely

In [2]:
import utils as U

In [3]:
query="""select * from site_reports where id =:proj_id"""
con,cur=U.connect_to_db()
cur.execute(query,{'proj_id': 1})

In [5]:
cols=[col[0] for col in cur.description]
cols

['id', 'site_name', 'lat', 'lng', 'location', '500_buffer', '15m_catchment']

In [6]:
data=cur.fetchone()
data

(1,
 None,
 13.011026921074786,
 77.55473637957641,
 None,
 'POLYGON ((77.55064773796063 13.010557348159388, 77.55062048378201 13.01095256882238, 77.5506328657994 13.011348506006348, 77.5506847650284 13.01174134701184, 77.55077568198409 13.012127308918089, 77.55090474147863 13.012502675000924, 77.55107070103462 13.012863830515565, 77.5512719628341 13.013207297499356, 77.55150658908862 13.013529768260184, 77.5517723206831 13.0138281372284, 77.55206659891435 13.014099530864803, 77.55238659011606 13.014341335338008, 77.55272921293349 13.014551221702861, 77.55309116798556 13.014727168338966, 77.55346896962965 13.014867480431064, 77.5538589795232 13.014970806304975, 77.55425744165935 13.015036150459899, 77.55466051853922 13.015062883171515, 77.55506432813272 13.015050746573092, 77.55546498127137 13.014999857154676, 77.55585861911337 13.01491070465632, 77.55624145031891 13.014784147365518, 77.55660978757814 13.014621403862282, 77.55696008313859 13.014424041293346, 77.55728896298986 13.014193

In [8]:
df=pd.DataFrame([dict(zip(cols,data))])
df

,id,site_name,lat,lng,location,500_buffer,15m_catchment
0,1,None,13.011027,77.554736,None,POLYGON ((77.55064773796063 13.010557348159388...,"POLYGON ((77.494736 13.050067, 77.493202 13.05..."


In [33]:
def get_population_of_intersection(pop, g1, g2):
    if not g2:
        return 0
    intersecting_area = (g1.intersection(g2).area / g2.area)
    return intersecting_area * pop


In [6]:
def get_population_index(poly):
    aff_ind = pd.read_csv("data/100m_grid_population.csv")
    cols = ["total_population", "f_0", "f_1", "f_10", "f_15", "f_20", "f_25", "f_30", "f_35", "f_40", "f_45", "f_5",
            "f_50", "f_55", "f_60", "f_65", "f_70", "f_75", "f_80", "m_0", "m_1", "m_10", "m_15", "m_20", "m_25",
            "m_30", "m_35", "m_40", "m_45", "m_5", "m_50", "m_55", "m_60", "m_65", "m_70", "m_75", "m_80"]
    aff_ind_gdf = gpd.GeoDataFrame(aff_ind,
                                   geometry=gpd.GeoSeries.from_wkt(aff_ind['geometry']))
    aff_ind_gdf['pop_grid_geometry'] = aff_ind_gdf['geometry']
    stores_gdf = gpd.GeoDataFrame([{"store_name": 1, "geometry": poly}], geometry='geometry')
    joined = gpd.sjoin(stores_gdf, aff_ind_gdf, how="left", predicate='intersects')
    for col in cols:
        joined[f'new_{col}'] = joined[[col, 'geometry', 'pop_grid_geometry']].apply(
            lambda x: get_population_of_intersection(x[0], x[1], x[2]), axis=1)
    return joined

In [8]:
poly = pd.read_csv("data/catchment.csv")['500_buffer'].values.tolist()[0]
poly = shapely.from_wkt(poly)
df = get_population_index(poly)


/tmp/ipykernel_49836/178425780.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: get_population_of_intersection(x[0], x[1], x[2]), axis=1)
/tmp/ipykernel_49836/178425780.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: get_population_of_intersection(x[0], x[1], x[2]), axis=1)
/tmp/ipykernel_49836/178425780.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: get_population_of_intersection(x[0], x

In [10]:
cols = ["total_population", "f_0", "f_1", "f_10", "f_15", "f_20", "f_25", "f_30", "f_35", "f_40", "f_45", "f_5", "f_50",
        "f_55", "f_60", "f_65", "f_70", "f_75", "f_80", "m_0", "m_1", "m_10", "m_15", "m_20", "m_25", "m_30", "m_35",
        "m_40", "m_45", "m_5", "m_50", "m_55", "m_60", "m_65", "m_70", "m_75", "m_80"]

In [13]:
fns = {col: (col, np.sum) for col in cols}
df.groupby("store_name").agg(**fns).to_dict('records')

{'total_population': ('total_population', <function sum at 0x7f4a1c3bd830>),
 'f_0': ('f_0', <function sum at 0x7f4a1c3bd830>),
 'f_1': ('f_1', <function sum at 0x7f4a1c3bd830>),
 'f_10': ('f_10', <function sum at 0x7f4a1c3bd830>),
 'f_15': ('f_15', <function sum at 0x7f4a1c3bd830>),
 'f_20': ('f_20', <function sum at 0x7f4a1c3bd830>),
 'f_25': ('f_25', <function sum at 0x7f4a1c3bd830>),
 'f_30': ('f_30', <function sum at 0x7f4a1c3bd830>),
 'f_35': ('f_35', <function sum at 0x7f4a1c3bd830>),
 'f_40': ('f_40', <function sum at 0x7f4a1c3bd830>),
 'f_45': ('f_45', <function sum at 0x7f4a1c3bd830>),
 'f_5': ('f_5', <function sum at 0x7f4a1c3bd830>),
 'f_50': ('f_50', <function sum at 0x7f4a1c3bd830>),
 'f_55': ('f_55', <function sum at 0x7f4a1c3bd830>),
 'f_60': ('f_60', <function sum at 0x7f4a1c3bd830>),
 'f_65': ('f_65', <function sum at 0x7f4a1c3bd830>),
 'f_70': ('f_70', <function sum at 0x7f4a1c3bd830>),
 'f_75': ('f_75', <function sum at 0x7f4a1c3bd830>),
 'f_80': ('f_80', <function 

In [19]:
df1 = (
    pd.read_csv("data/bng_school_colleges.csv")
    .dropna(subset=["id"])
    .drop_duplicates(subset=["id"])
)
df2 = pd.read_csv("data/bng_school_colleges_lat_lng.csv")[['id', 'lat', 'lng']]
mask = df1["category"].isin(["college", "school", ]) & ~df1["verified"].isin(['FALSE'])
df = df1[mask]
df = df.merge(df2, on="id", how="inner")

In [21]:
df.to_csv("data/bng_school_colleges_final.csv", index=False)

In [22]:
cols = ['id', 'name', 'lat', 'lng', 'category', 'number_of_votes', 'source', 'date', 'created_at']
df1 = pd.read_csv("data/non_branded_data_zomato_oldest_ts.csv")[cols]
df2 = pd.read_csv("data/non_branded_data_google_oldest_ts.csv")[cols]
df = pd.concat([df1, df2]).drop_duplicates(subset=['id']).reset_index(drop=True)
df = df.fillna({"number_of_reviews": 0, "date": 0})

In [23]:
df['reviews_per_day'] = df['number_of_votes'] / (df['created_at'] - df['date'])

In [25]:
df.sort_values(by='reviews_per_day', ascending=False).to_csv("data/non_branded_data_ranked.csv", index=False)

In [57]:
poly = pd.read_csv("data/catchment.csv")['500_catchment'].values.tolist()[0]
poly = shapely.from_wkt(poly)
# df=get_population_index(poly)


In [58]:
aff_ind = pd.read_csv("data/bng_malls.csv")
aff_ind_gdf = gpd.GeoDataFrame(aff_ind, geometry=gpd.GeoSeries.from_wkt(aff_ind['wkt']))
# aff_ind_gdf['geom']=aff_ind_gdf['geom'].apply(shapely.from_wkt)
stores_gdf = gpd.GeoDataFrame([{"store_name": 1, "geometry": poly}], geometry='geometry')
gpd.sjoin(stores_gdf, aff_ind_gdf, predicate='intersects')


,store_name,geometry,index_right,id,name,wkt
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",12,ChIJbQb8IPs9rjsR4j-RbzJFroA,GT World Mall,POLYGON ((77.55169820246995 12.974161619340574...
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",14,ChIJlb8QkIwXrjsRGUxSwTzIMhQ,Lulu Mall,POLYGON ((77.56289552267329 12.982497418607135...
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",13,ChIJEVTX0_U9rjsRrLNuiOLZgbs,Golden Heights,POLYGON ((77.55897393827507 12.983558968292243...
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",11,ChIJLfcwwb89rjsRuEH5q3deUfQ,Platinum Mall,"POLYGON ((77.53784923874949 12.98897115145576,..."
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",10,ChIJoRKfACMWrjsRrWqiP9PBT5Y,Mantri Mall,"POLYGON ((77.56961331170093 12.99219738364626,..."
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",9,ChIJZ0FegEs9rjsRZFR-C-hVwqQ,Brigade Orion Mall,POLYGON ((77.55455350014815 13.012127190197445...
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",8,ChIJCUMusVM9rjsREFoNQDA52s0,METRO Wholesale,POLYGON ((77.55424999112478 13.015306328747194...
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",7,ChIJj9VQyNE9rjsRsVrZQsVMFmM,Vaishnavi Sapphire Mall,POLYGON ((77.54833380341793 13.025176488534848...
0,1,"POLYGON ((77.49474 13.05007, 77.49320 13.05003...",62,ChIJez6EsCc9rjsR1OgmK7Vjxvs,Rock Line Mall,POLYGON ((77.51818646390602 13.041153773888459...


In [65]:
malls = pd.read_csv("data/bng_malls_ranked.csv")
malls = gpd.GeoDataFrame(malls, geometry=gpd.GeoSeries.from_wkt(malls['wkt']))
stores_gdf = gpd.GeoDataFrame([{"store_name": 1, "geometry": poly}], geometry='geometry')
joined = gpd.sjoin(stores_gdf, malls, how="left", predicate='intersects')

In [66]:
joined = joined.sort_values(by=['reviews_per_day'], ascending=False)
joined = joined[['id', 'name', 'number_of_votes', 'ratings', 'lat', 'lng']]

In [63]:
centroid = poly.centroid
lat = centroid.y
lng = centroid.x

In [67]:
from geopy.distance import geodesic


In [75]:
joined['distance'] = joined[['lat', 'lng']].apply(lambda x: round(geodesic((x[0], x[1]), (lat, lng)).km, 2), axis=1)

/tmp/ipykernel_8127/3216831717.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  joined['distance']=joined[['lat','lng']].apply(lambda x:round(geodesic((x[0],x[1]),(lat,lng)).km,2),axis=1)


In [102]:
stores

,lat,lng,500_buffer,500_catchment,store_name
0,13.011027,77.554736,POLYGON ((77.55064773796063 13.010557348159388...,"POLYGON ((77.494736 13.050067, 77.493202 13.05...",1
1,NaN,NaN,NaN,NaN,1


In [164]:
from functools import reduce

df1 = pd.read_csv('data/non_branded_data_ranked.csv').dropna(subset=['id']).drop_duplicates(subset=['id'])
df1['brand_id'] = 'N_A'
df1['brand_name'] = 'N_A'
df2 = pd.read_csv('data/bng_branded_data_ranked.csv').dropna(subset=['id']).drop_duplicates(subset=['id'])
df = pd.concat([df1, df2])

In [165]:
df['brand_name'] = df['brand_name'].str.title()

In [169]:
"N_A".title()

'N_A'

In [156]:
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lng'], df['lat']))
stores = pd.read_csv("data/catchment.csv").dropna(subset=['lat'])
stores['store_name'] = 1
geometry_cols = ['500_buffer', '500_catchment']
# geometry_cols=['500_buffer']
joined_dfs = {}
for geometry_col in geometry_cols:
    stores_gdf = gpd.GeoDataFrame(
        stores, geometry=gpd.GeoSeries.from_wkt(stores[geometry_col])
    )
    joined_ = stores_gdf.sjoin(df_gdf, how="inner", predicate="contains")
    joined_['geometry_type'] = geometry_col
    joined1 = joined_.groupby(["store_name", "category"]).agg(
        **{f"count": ("id", np.count_nonzero)}
    ).reset_index()
    joined2 = joined_[joined_["brand_id"] != 'N_A'].groupby(["store_name", "category", 'brand_name']).agg(
        **{f"ranking": ("reviews_per_day", np.sum),
           }
    ).reset_index().groupby(["category", ])[['brand_name', 'ranking']].apply(
        lambda g: dict(map(tuple, g.values.tolist()))).reset_index().rename(columns={0: 'top_brands'})
    joined = joined1.merge(joined2, how="left", on='category')
    joined = joined.reset_index(drop=True)
    joined = joined[['category', 'count', 'top_brands']].to_dict(orient='records')
    joined_dfs[geometry_col] = joined
# branded_stores=reduce(lambda a,b:pd.merge(a,b,how='left',on='store_name') ,joined_dfs).fillna(0)
# branded_stores.shape

/tmp/ipykernel_8127/4274979533.py:23: FutureWarning: The provided callable <function sum at 0x7f3e7078af80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  joined2=joined_[joined_["brand_id"]!='N_A'].groupby(["store_name", "category",'brand_name']).agg(
/tmp/ipykernel_8127/4274979533.py:23: FutureWarning: The provided callable <function sum at 0x7f3e7078af80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  joined2=joined_[joined_["brand_id"]!='N_A'].groupby(["store_name", "category",'brand_name']).agg(


In [248]:
df1 = pd.read_csv('data/non_branded_data_ranked.csv').dropna(subset=['id']).drop_duplicates(subset=['id'])
df1['brand_id'] = 'N_A'
df1['brand_name'] = 'N_A'
df2 = pd.read_csv('data/bng_branded_data_ranked.csv').dropna(subset=['id']).drop_duplicates(subset=['id'])
df = pd.concat([df1, df2])
df['brand_name'] = df['brand_name'].str.title()

df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lng'], df['lat']))
stores = pd.read_csv("data/catchment.csv").dropna(subset=['lat']).drop(columns=['lat','lng'])
stores['store_name'] = 1
geometry_cols = ['500_buffer', '500_catchment']
joined_dfs = {}
for geometry_col in geometry_cols:
    stores_gdf = gpd.GeoDataFrame(
        stores, geometry=gpd.GeoSeries.from_wkt(stores[geometry_col])
    )
    joined_ = stores_gdf.sjoin(df_gdf, how="inner", predicate="contains")
    # pois=
    joined_['geometry_type'] = geometry_col
    joined1 = joined_.groupby(["store_name", "category"]).agg(
        **{f"count": ("id", np.count_nonzero)}
    ).reset_index()
    joined2 = joined_[joined_["brand_id"] != 'N_A'].groupby(["store_name", "category", 'brand_name']).agg(
        **{f"ranking": ("reviews_per_day", np.sum),
           }
    ).reset_index().groupby(["category", ])[['brand_name', 'ranking']].apply(
        lambda g: dict(map(tuple, g.values.tolist()))).reset_index().rename(columns={0: 'top_brands'})
    joined = joined1.merge(joined2, how="left", on='category')
    joined = joined.reset_index(drop=True)
    joined = joined[['category', 'count', 'top_brands']].to_dict(orient='records')
    joined_dfs[geometry_col] = joined

/tmp/ipykernel_8127/3513534042.py:23: FutureWarning: The provided callable <function sum at 0x7f3e7078af80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  joined2 = joined_[joined_["brand_id"] != 'N_A'].groupby(["store_name", "category", 'brand_name']).agg(
/tmp/ipykernel_8127/3513534042.py:23: FutureWarning: The provided callable <function sum at 0x7f3e7078af80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  joined2 = joined_[joined_["brand_id"] != 'N_A'].groupby(["store_name", "category", 'brand_name']).agg(


In [250]:
df_gdf

,id,name,lat,lng,category,number_of_votes,source,date,created_at,reviews_per_day,brand_id,brand_name,geometry
0,ChIJQXJsfNE9rjsR1EmZaZq1D94,Qmin - Orion Mall,13.010914,77.555061,coffee_shop,95,gmaps_v2,1.687675e+09,1687847039,0.000551,N_A,N_A,POINT (77.55506 13.01091)
1,ChIJ53uzJFI9rjsRsk2X1pPn_jc,"Maiara ( Women""s couture)",12.991986,77.538696,clothing_store,53,gmaps_v2,1.687697e+09,1687847039,0.000352,N_A,N_A,POINT (77.53870 12.99199)
2,ChIJTxP0xoUXrjsR67mJ7s39hi4,Badmaash Lulu Mall,12.981327,77.563232,restaurant,31,gmaps_v2,1.699514e+09,1699615207,0.000308,N_A,N_A,POINT (77.56323 12.98133)
3,ChIJqc6_h1kXrjsRz_rsaXasik4,SriStar Gold Company,13.008282,77.571259,jewelry_store,1902,gmaps_v2,1.680940e+09,1687847039,0.000275,N_A,N_A,POINT (77.57126 13.00828)
4,ChIJ82NqvCUWrjsRC33A4yTYQWk,Central Tiffin Room,12.998143,77.569557,restaurant,31067,gmaps_v2,1.576297e+09,1699615207,0.000252,N_A,N_A,POINT (77.56956 12.99814)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,ChIJ0X5qOsEXrjsR58Lg9Up-ka4,The Sleep Company Experience Store - Seshadrip...,12.991714,77.570671,home_decor,165,gmaps_v2,1.697905e+09,1695212099,-0.000061,thesleepcompany,The Sleep Company,POINT (77.57067 12.99171)
1160,ChIJ70E8-L09rjsRaF25s2D6dLU,Levi's Exclusive Store,12.993377,77.538879,clothing_store,198,gmaps_v2,1.690990e+09,1687847039,-0.000063,levi,Levi,POINT (77.53888 12.99338)
1161,ChIJ3eYZLvo9rjsRHn26qEQpZRM,Max,12.989475,77.537582,clothing_store,64,gmaps_v2,1.688439e+09,1687847039,-0.000108,maxfashion,Max Fashion,POINT (77.53758 12.98948)
1162,ChIJs24k0u4XrjsRdLiCiYwzZ1M,Blackberrys,12.979923,77.559929,clothing_store,102,gmaps_v2,1.688728e+09,1687847039,-0.000116,blackberrys,Blackberrys,POINT (77.55993 12.97992)


In [258]:
g=joined_[['id','name','lat','lng','source','category','number_of_votes','reviews_per_day']].groupby(['category'])

In [277]:
dict_list=[]
for name, group in g:
    d= {'category': name[0], 'pois': group.to_dict('records')}
    dict_list.append(d)
top_pois=pd.DataFrame(dict_list)

In [278]:
pd.DataFrame(dict_list)

,category,pois
0,clothing_store,"[{'id': 'ChIJr9vHZDU9rjsRw0jHeoMWzb0', 'name':..."
1,coffee_shop,"[{'id': 'ChIJ6RWvxOM9rjsRJSbElIUDP2E', 'name':..."
2,home_decor,"[{'id': 'ChIJi63UT0Q9rjsRhcDSd80c-jQ', 'name':..."
3,jewelry_store,"[{'id': 'ChIJZ4ObfuE9rjsRRhpeV9vkdnc', 'name':..."
4,restaurant,"[{'id': 'ChIJtcUulho-rjsRkz7nQ-vL0Yg', 'name':..."
5,shoe_store,"[{'id': 'ChIJF_rAstU_rjsREsu01_WjXao', 'name':..."
6,shopping_mall,"[{'id': 'ChIJ83WkACMWrjsRpQaheGm8VaE', 'name':..."
7,supermarket,"[{'id': 'ChIJtffXKeg9rjsRynId3oTBy5M', 'name':..."


In [183]:
joined_[['id', 'name', 'lat', 'lng', 'category', 'source', 'number_of_votes', 'reviews_per_day']].sort_values(
    by='reviews_per_day', ascending=False).reset_index(drop=True).to_csv("data/top_pois_15_min_catchment.csv",
                                                                         index=True)

In [130]:
joined_[joined_["brand_id"] != 'N_A'].groupby(["store_name", "category", 'brand_name']).agg(
    **{f"ranking": ("reviews_per_day", np.sum),
       }
).reset_index().groupby(["category", ])[['brand_name', 'ranking']].apply(
    lambda g: dict(map(tuple, g.values.tolist()))).reset_index()


/tmp/ipykernel_8127/2427305067.py:1: FutureWarning: The provided callable <function sum at 0x7f3e7078af80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  joined_[joined_["brand_id"]!='N_A'].groupby(["store_name", "category",'brand_name']).agg(


,category,0
0,clothing_store,"{'AND India': 3.1021283849792265e-07, 'Allen S..."
1,coffee_shop,"{'Amrut Chaha': 0.0, 'Cafe Coffee Day': 4.5238..."
2,home_decor,"{'Homecentre': 8.022256590329601e-06, 'Nilkama..."
3,jewelry_store,"{'CaratLane': 4.747157161851056e-05, 'Kushal's..."
4,restaurant,"{'Aaha Andhra': 1.9607377740260042e-07, 'Baski..."
5,shoe_store,"{'ALDO': 1.1855933869842727e-07, 'ASICS': 1.00..."


In [140]:
joined_[joined_["brand_id"] != 'N_A'].groupby(["store_name", "category", 'brand_name']).agg(
    **{f"ranking": ("reviews_per_day", np.sum),
       }
).reset_index().groupby(["category", ])[['brand_name', 'ranking']].apply(
    lambda g: dict(map(tuple, g.values.tolist()))).reset_index().rename(columns={0: 'top_brands'})


/tmp/ipykernel_8127/3097969619.py:1: FutureWarning: The provided callable <function sum at 0x7f3e7078af80> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  joined_[joined_["brand_id"]!='N_A'].groupby(["store_name", "category",'brand_name']).agg(


,category,top_brands
0,clothing_store,"{'AND India': 3.1021283849792265e-07, 'Allen S..."
1,coffee_shop,"{'Amrut Chaha': 0.0, 'Cafe Coffee Day': 4.5238..."
2,home_decor,"{'Homecentre': 8.022256590329601e-06, 'Nilkama..."
3,jewelry_store,"{'CaratLane': 4.747157161851056e-05, 'Kushal's..."
4,restaurant,"{'Aaha Andhra': 1.9607377740260042e-07, 'Baski..."
5,shoe_store,"{'ALDO': 1.1855933869842727e-07, 'ASICS': 1.00..."


In [87]:
df1 = pd.read_csv("data/raw/branded_data_ranked.csv")
df2 = pd.read_csv("data/raw/bng_branded_data.csv").drop_duplicates(subset=['id'])[['id', 'brand_id', 'brand_name']]

In [92]:
malls = pd.read_csv("data/bng_malls_ranked.csv")
df1 = df1[~df1['id'].isin(malls['id'].values.tolist())]

In [97]:
df1.merge(df2, how='left', on='id').to_csv("data/raw/bng_branded_data_ranked.csv", index=False)

In [229]:
def get_trans_type(desc, fallback):
    if fallback.lower() in ('buy', 'sale', 'resale', 'new property') or 'sale' in desc.lower():
        return 'buy'
    elif fallback.lower() == 'rent' or 'rent' in desc.lower():
        return 'rent'
    else:
        return fallback


def get_prop_type(desc, fallback):
    if 'apartment' in desc.lower() or 'apartment' in fallback.lower():
        return 'AP'
    elif 'flat' in desc.lower():
        return 'AP'
    else:
        return fallback

In [243]:
df = pd.read_csv("data/raw/bng_properties.csv")
df = df.fillna({"price": 0, 'type': '', 'trans_type': '', 'desc': '', 'name': ''})
df['trans_type'] = df[['desc', 'trans_type']].apply(lambda x: get_trans_type(x[0], x[1]), axis=1)
df['type'] = df[['name', 'type']].apply(lambda x: get_prop_type(x[0], x[1]), axis=1)
mask = (df['type'] == 'AP') & (df['trans_type'].isin(['buy', 'rent']))
df = df[mask]
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lng'], df['lat']))
stores = pd.read_csv("data/catchment.csv").dropna(subset=['lat'])
stores['store_name'] = 1
geometry_cols = ['500_buffer', '500_catchment']
# geometry_cols = [ '500_catchment']
joined_dfs = {}
for geometry_col in geometry_cols:
    stores_gdf = gpd.GeoDataFrame(
        stores, geometry=gpd.GeoSeries.from_wkt(stores[geometry_col])
    )
    joined_ = stores_gdf.sjoin(df_gdf, how="inner", predicate="contains")
    joined_['geometry_type'] = geometry_col
    joined = joined_.groupby(["store_name", "trans_type"]).agg(
        **{f"median_price": ("price", np.median), 
           # f"mean_price": ("price", np.mean)
           }
    ).reset_index().drop(columns=['store_name']).to_dict(orient='records')[0]
    joined_dfs[geometry_col] = joined


/tmp/ipykernel_8127/3827802176.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['trans_type'] = df[['desc', 'trans_type']].apply(lambda x: get_trans_type(x[0], x[1]), axis=1)
/tmp/ipykernel_8127/3827802176.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['type'] = df[['name', 'type']].apply(lambda x: get_prop_type(x[0], x[1]), axis=1)
/tmp/ipykernel_8127/3827802176.py:19: FutureWarning: The provided callable <function median at 0x7f3e701e2440> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "media

In [244]:
joined_dfs

{'500_buffer': {'trans_type': 'buy', 'median_price': 46070152.0},
 '500_catchment': {'trans_type': 'buy', 'median_price': 28900000.0}}

In [241]:
df['prop_type_new'].value_counts()

KeyError: 'prop_type_new'

In [198]:
joined_['trans_type'].value_counts()

trans_type
Buy             1143
New Property    1088
Resale           998
Rent               3
Name: count, dtype: int64

In [279]:
df = (
        pd.read_csv("data/raw/hospitals_clinic.csv")
        .dropna(subset=["id"])
        .drop_duplicates(subset=["id"])
    )
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lng"], df["lat"]))
stores_gdf = gpd.GeoDataFrame([{"store_name": 1, "geometry": poly}], geometry='geometry')
joined_ = stores_gdf.sjoin(df_gdf, how="inner", predicate="contains")
g=joined_[['id', 'name', 'lat', 'lng', 'source', 'category', 'number_of_votes']].groupby(["category"])
dict_list=[]
for name, group in g:
    d = {'category': name[0], 'top_pois': group.to_dict('records')}
    dict_list.append(d)
top_pois_by_category = pd.DataFrame(dict_list)
geometry_col = "geometry"
joined = (
    joined_.groupby(["store_name", "category"])
    .agg(**{f"{geometry_col}_type": ("id", np.count_nonzero)})
    .reset_index()
    .pivot(
        index="store_name",
        columns="category",
        values=f"{geometry_col}_type",
    )
)
co=joined.to_dict('records')[0]
resp={}
resp["count"]=co
resp['top_pois']=top_pois_by_category


In [285]:
top_pois_by_category.to_dict(orient='records')[0]

{'category': 'clinic',
 'top_pois': [{'id': 'ChIJlYdXh-E9rjsRXxKpM2Gq4D8',
   'name': 'Sai Dental Care Center',
   'lat': 12.967217445373535,
   'lng': 77.54473876953125,
   'source': 'gmaps_v2',
   'category': 'clinic',
   'number_of_votes': 11},
  {'id': 'ChIJ_1MTrCM9rjsR7uYVTkeNZYk',
   'name': 'Acupunture & Acupressure Multiple Therapy Center',
   'lat': 12.967265129089355,
   'lng': 77.54489135742188,
   'source': 'gmaps_v2',
   'category': 'clinic',
   'number_of_votes': 28},
  {'id': 'ChIJxyjP6p09rjsRoBlK7HVotsA',
   'name': 'Plexihealth Labs- Orthopaedics and Skin Centre',
   'lat': 12.967324256896973,
   'lng': 77.54468536376953,
   'source': 'gmaps_v2',
   'category': 'clinic',
   'number_of_votes': 28},
  {'id': 'ChIJq9dK-VQ9rjsR-Vg2RNrf5V4',
   'name': 'MIARA clinic - Ortho | Physio | Skin',
   'lat': 12.967324256896973,
   'lng': 77.5446548461914,
   'source': 'gmaps_v2',
   'category': 'clinic',
   'number_of_votes': 9},
  {'id': 'ChIJ2ZziYNc9rjsRgIt6nWfffMM',
   'name': 

In [10]:
import requests
import json

In [12]:
eps=["cft","population","companies","education","projects","affluence","income","competition","category_count","property_price","medical","high_street",]

In [15]:
for ep in eps:
    url=f"http://127.0.0.1:8000/site_report/{ep}"
    req=requests.get(url)
    resp=req.json()
    with open(f"data/api_response/{ep}.json",'w') as f:
        json.dump(resp,f,indent=4)
        

In [15]:
import importlib
import dashboard as d
importlib.reload(d)

<module 'dashboard' from '/home/jyotiraditya/PycharmProjects/SiteReports/dashboard.py'>

In [16]:
man

AttributeError: module 'utils' has no attribute 'get_project_info'

In [1]:
import osmnx as ox

In [9]:
tags = {"boundary":"administrative" }
tags={'admin_level':"4"}
ox.features_from_point((13.0552603,	77.7638939),tags=tags,dist=50000)

/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)
/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)
/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [15]:
ox.geocode_to_gdf("bengaluru",)

/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will raise in a future version. Use public APIs instead.
  srs = pd.Series(*args, **kwargs)
/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/pandas/core/frame.py:706: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/home/jyotiraditya/PycharmProjects/SiteReports/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1645: DeprecationWarning: Passing a SingleBlockManager to Series is deprecated and will rai

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((77.46010 12.90384, 77.46022 12.90045...",13.143665,12.834013,77.784052,77.460103,377749295,relation,7902476,12.976794,77.590082,boundary,administrative,16,0.609435,city,Bengaluru,"Bengaluru, Bangalore North, Bengaluru Urban Di..."


In [16]:
from geopy.distance import  geodesic

In [ ]:
geodesic()